## 操作方式和 DataClass一樣

In [1]:
from pydantic import BaseModel

class Person(BaseModel):  
    # field 欄位一定要有type hint
    first_name:str
    last_name:str
    age:int

p = Person(first_name="Jone", last_name="Smith", age=42)
print(p.first_name)
print(p.last_name)
print(p.age)
print(p)

Jone
Smith
42
first_name='Jone' last_name='Smith' age=42


## dataclass做不到的
- ### 驗證資料

In [2]:
p1 = Person(first_name="Jone", last_name="Smith", age="42") #自動轉換型別
print(p1)

first_name='Jone' last_name='Smith' age=42


In [4]:
try:
    p2 = Person(first_name="Jone", last_name="Smith", age="abc") #自動轉換型別
except Exception as error:
    print(error)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


## 可以將純文字轉成pydantic實體
- # model_validate_json()

In [5]:
data_json = '''   # json不能用單引號''
{
    "first_name":"john",
    "last_name":"Smith",
    "age":"42"
}
'''
p3 = Person.model_validate_json(data_json)
print(p3)

first_name='john' last_name='Smith' age=42


## 可以選擇需要的欄位

In [10]:
data_json = '''     
{
    "first_name":"john",
    "last_name":"Smith",
    "age":42,
    "height":173
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='john' last_name='Smith' age=42


## 可以設定取得的資料,如果沒有這個欄位,設定欄位預設值

In [12]:
class Person(BaseModel):  
    # field 欄位一定要有type hint
    first_name:str #一定要有資料
    last_name:str #一定要有資料
    age:int = 0 #有預設的值
    lucky_number:list[int] = []

data_json = '''     
{
    "first_name":"john",
    "last_name":"Smith",
    "lucky_number":[1,2,3,4,"5"]
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='john' last_name='Smith' age=0 lucky_number=[1, 2, 3, 4, 5]


In [13]:
data_json = '''     
{
    "id":100,
    "First Name":"Smith",
    "LASTNAME":"Smith"
    "age in years":42
}
'''

from pydantic import BaseModel, Field

class Person(BaseModel):
    id_:int = Field(alias="id")
    first_name:str = Field(alias="First Name")
    last_name:str = Field(alias="LASTNAME")
    age:int = Field(alias="age in years",default=0)

    p1 = Person.model_validate_json(data_json)
    print(p1)

ValidationError: 1 validation error for Person
  Invalid JSON: expected `,` or `}` at line 6 column 5 [type=json_invalid, input_value='     \n{\n    "id":100,\... "age in years":42\n}\n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/json_invalid

In [18]:
data:str = '''
{
    "firstName": "Arthur",
    "lastName": "Clarke",
    "born":{
        "place":{
            "country":"Lunar Colony",
            "city": "Central City"
        },
        "date":"2001-01-01"
    }
}
'''
from pydantic import BaseModel, Field
class Place(BaseModel):
    country:str
    city:str

class Born(BaseModel):
    place:Place
    date:str

class Person(BaseModel):
    first_name:str = Field(alias="firstName")
    last_name:str = Field(alias="lastName")
    born:Born

p1 = Person.model_validate_json(data)
print(p1.born.place.country)
print(p1.born.place.city)

Lunar Colony
Central City
